In [1]:
import math
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [2]:
class Value:
  def __init__(self, data, _children=(), _op=''):
    self.data = data
    self.grad = 0.0
    self._backward = lambda: None
    self._prev = set(_children)
    self._op = _op

  def backward(self):
    # Implements backward pass or backpropagation
    topo = []
    visited = set()
    def build_topo(v):
      if v not in visited:
        visited.add(v)
      for child in v._prev:
        build_topo(child)
      topo.append(v)
    
    build_topo(self)

    self.grad = 1
    for v in reversed(topo):
      v._backward()
    

  def __add__(self, other):
    other = Value(other) if not isinstance(other, Value) else other
    out = Value(self.data + other.data)
    def _backward():
      self.grad += out.grad
      other.grad += out.grad
    out._backward = _backward
    return out


  def __radd__(self, other):
    return self + other

  def __mul__(self, other):
    other = Value(other) if not isinstance(other, Value) else other
    out = Value(self.data * other.data)
    def _backward():
      self.grad += other.data * out.grad
      other.grad += self.data * out.grad
    out._backward = _backward
    return out

  def __rmul__(self, other):
    return self * other

  def __neg__(self):
    return Value(-1 * self.data)

  def __sub__(self, other):
    return self + (-other)

  def __rsub__(self, other):
    return other + (-self)

  def __pow__(self, pow):
    out = Value(self.data ** pow)
    def _backward():
      self.grad += pow * (self.data ** (pow-1)) * out.grad
    out._backward = _backward
    return out

  def __truediv__(self, other):
    return self * (other ** -1)

  def __rtruediv__(self, other):
    return other * (self ** -1)

  def relu(self):
    out = Value(0 if self.data < 0 else self.data)
    def _backward():
      self.grad += (out.data > 0) * out.grad
    out._backward = _backward
    return out

  def __repr__(self):
    return f"Value: {self.data}, grad: {self.grad}"

class NNBaseClass:
    
  def parameters(self):
    return []

  def zero_grad(self):
    for parameter in self.parameters():
      parameter.grad = 0

